In [ ]:
import requests
import json

# URL 설정
url = "https://parking.seoul.go.kr/SearchParking.do"

# 헤더 설정
headers = {
    "accept": "*/*",
    "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Mobile Safari/537.36",
    "x-requested-with": "XMLHttpRequest"
}


# 위도, 경도, 반경 설정
payload = {
    "LAT": "37.5663",
    "LON": "126.9911",
    "range": 500
}

# POST 요청 보내기
response = requests.post(url, headers=headers, data=payload, verify=False)

# 데이터 저장 변수
parking_data = []

# 응답 데이터 출력 및 파싱
if response.status_code == 200:
    response_data = response.json()  # JSON으로 변환
    parking_list = response_data.get("res_value", {}).get("parking_list", [])

    # 주차장 리스트 저장
    for parking in parking_list:
        try:
            capacity = int(parking.get('capacity', 0))
            cur_parking = int(parking.get('cur_parking', 0))
            available_parking = capacity - cur_parking
        except ValueError:
            available_parking = None  # 주차 가능 면을 계산할 수 없는 경우 None 처리

        parking_info = {
            "주차장 이름": parking.get('parking_name'),
            "주소": parking.get('address'),
            "새 주소": parking.get('new_juso'),
            "주차 타입": parking.get('parking_type'),
            "전체 주차 면": capacity,
            "현재 주차 차량 수": cur_parking,
            "주차 가능 면": available_parking,
            "요금": f"{parking.get('rates')}원/{parking.get('time_rate')}분",
            "추가 요금": f"{parking.get('add_rates')}원",
            "요금 부과 기준": f"{parking.get('time_rate')}분",
            "정기권 요금": f"{parking.get('fulltime_monthly')}원",
            "일일 최대 요금": f"{parking.get('day_maximum')}원",
            "버스 요금": parking.get('bus_rates'),
            "버스 추가 요금": parking.get('bus_add_rates'),
            "버스 시간당 요금": parking.get('bus_time_rate'),
            "버스 추가 시간당 요금": parking.get('bus_add_time_rate'),
            "현재 주차 갱신 시간": parking.get('cur_parking_time'),
            "평일 운영 시간": f"{parking.get('weekday_begin_time')} ~ {parking.get('weekday_end_time')}",
            "주말 운영 시간": f"{parking.get('weekend_begin_time')} ~ {parking.get('weekend_end_time')}",
            "공휴일 운영 시간": f"{parking.get('holiday_begin_time')} ~ {parking.get('holiday_end_time')}",
            "전화번호": parking.get('phone'),
            "연계 상태": parking.get('que_status_nm'),
            "위치 정보": [(position.get('lat'), position.get('lng')) for position in parking.get('position_list', [])]
        }

        # 리스트에 저장
        parking_data.append(parking_info)

else:
    print(f"요청 실패. 상태 코드: {response.status_code}")

# 저장된 데이터를 출력하는 함수
def print_parking_data():
    for parking in parking_data:
        for key, value in parking.items():
            print(f"{key}: {value}")
        print()  # 각 주차장 정보 사이에 빈 줄 추가

# 주차 정보 출력
print_parking_data()